# use GSEAPY for gsea analysis
## manual 
* https://buildmedia.readthedocs.org/media/pdf/gseapy/latest/gseapy.pdf

In [20]:
import pandas as pd
import glob, os, subprocess, sys, logging
import gseapy as gp

from gseapy.plot import gseaplot

from alive_progress import alive_bar

wk_dir = '/Users/jplab/Desktop/DAILY_CODE_DATA/2022-5/data/5-6_AT1_AT2_GSEA/'
os.chdir(wk_dir)

## make counts data to gsea expression txt format:
* reps **do not** need to be merged by mean value;
* column0 gene symbol; column1 descrpition(use NA to fill); column2 deseq2 normalized counts

In [21]:
deg_for_gsea_file_path = '/Users/jplab/Desktop/DAILY_CODE_DATA/2022-3/data/3-9_lysosome_gseapy/*txt'

deg_files = glob.glob(deg_for_gsea_file_path)

cls_file_list = [
    '/Users/jplab/Desktop/DAILY_CODE_DATA/2022-3/data/3-9_lysosome_gseapy/shNC_TGF_shNC_B14.cls',
    '/Users/jplab/Desktop/DAILY_CODE_DATA/2022-3/data/3-9_lysosome_gseapy/shFOXA1B14_shNCB14.cls',
    '/Users/jplab/Desktop/DAILY_CODE_DATA/2022-3/data/3-9_lysosome_gseapy/shNC_TGF_shFOXA1_TGF.cls',
    '/Users/jplab/Desktop/DAILY_CODE_DATA/2022-3/data/3-9_lysosome_gseapy/shFOXA1_TGF_shFOXA1_B14.cls'
]

### Note!
* **don't** use `.cls` file; can not get any results!!!
* use a string list instead!!! see below

In [ ]:
cls_list_order = []

# df_deg_list = []

for f in deg_files:
    f_name = os.path.basename(f)
    cls_list_order.append(f_name)

    # df_deg = pd.read_csv(f, header=0, sep='\t').drop_duplicates(subset=['NAME'])

    # df_deg_list.append(df_deg)
    
print('please provide cls file list, the order need to be same as: ',cls_list_order)
# print(df_deg_list)

* seems cls data cloud also accept a list of string, instead of a .cls file;

```python
# phenotype data
cls_b_d =['B14', 'B14','DMSO','DMSO']
cls_b_t =['B14', 'B14', 'TGF', 'TGF']
```

In [22]:
def do_gseapy(deg_file_list, gmt_file,min_gene_set_size,max_gene_set_size):
    # print('step into gesapy....')

    # print('using gmt : {0}'.format(gmt_file))

    gsea_result_path = wk_dir
    statu_flag = False

    for index, f_deg in enumerate(deg_files):

        f_name = os.path.basename(f_deg).split('_gseapy_input.txt')[0]
        df_deg = pd.read_csv(f_deg, header=0, sep='\t').drop_duplicates(subset=['NAME'])
        gmt_file_name = os.path.basename(gmt_file)
        # check gmt file is a temp file or not; if is a temp file, then remove prefix 'temp_' and suffix xx
        # an temp example: temp_c5.all.v7.5.1.symbols.gmtam

        if gmt_file_name.startswith('temp_'): 
            gmt_file_name = gmt_file_name.split('_')[-1]
            # print('gmt name use for fold: {0}'.format(gmt_file_name))

        # this_cls = cls_file_list[index]
        this_cls = df_deg.columns.values.tolist()

        # 1 is the maximum number of splits to perform
        this_cls = [i.split('_1', 1)[0] for i in this_cls]
        this_cls = [i.split('_2', 1)[0] for i in this_cls]
        this_cls = this_cls[2:]
        print('this_cls: {0}, using gmt file: {1}, using input: {2}'.format(this_cls, gmt_file_name, f_name))

        print(f'deg file name:{df_deg}, cls file name: {this_cls}')

        # codes for debug
        # with open(this_cls) as c:
        #     file = c.readlines()
        #     sample_name = file[1].lstrip("# ").strip('\n').split(" ")
        # print(sample_name)
        # print(len(sample_name))

        try:
            gs_res = gp.gsea(data=df_deg,
                            gene_sets=gmt_file,
                            min_size=min_gene_set_size,
                            max_size=max_gene_set_size,
                            cls=this_cls,
                            outdir=gsea_result_path+gmt_file_name+'/' + f_name + '/',
                            no_plot=False,
                            # set permutation_type to phenotype if samples >=15
                            permutation_type='gene_set',
                            method='log2_ratio_of_classes',
                            permutation_num=1000,
                            processes=16, seed=7,
                            format='png',
                            figsize=[6.5, 6]
                            )
            statu_flag = True                
        except Exception as e:
            statu_flag = False
            print(str(e))
            sys.exit('Oh no, Barbecue le....')

    return gs_res, statu_flag            

## add some config to GSEA

In [23]:
min_gene_set_size = 10
max_gene_set_size = 2000

In [25]:
# multifile
# gmt_file_path = '/Users/jplab/Desktop/freq_used_file/gene_sets/gmt/'
# gmt_files = glob.glob(gmt_file_path+'c*.v7.5.1.symbols.gmt')

# single file
# gmt_files = glob.glob('/Users/jplab/Desktop/freq_used_file/gene_sets/gmt/*.symbols.gmt')
gmt_files = glob.glob('/Users/jplab/Desktop/freq_used_file/gene_sets/gmt/AT*_Reyfman_et_al_2019.gmt')

# /Users/jplab/Desktop/freq_used_file/gene_sets/gmt/c5.all.v7.5.1.symbols.gmt
# /Users/jplab/Desktop/freq_used_file/gene_sets/gmt/c6.all.v7.5.1.symbols.gmt

for gmt_file in gmt_files:

    file_name = os.path.basename(gmt_file).split('_geneset.gmt')[0]

    # check how many lines in gmt file, if more than 2000, split it by 2000; or GSEApy will crash;
    num_lines = sum(1 for line in open(gmt_file))
    each_split_num = 100
    temp_file_format = 'temp_'+file_name

    if num_lines <= each_split_num:
        # print('gmt file: {0} is less than {1}, doing GSEApy now!'.format(gmt_file, each_split_num))

        gsea_res, status =  do_gseapy(deg_files, gmt_file,min_gene_set_size,max_gene_set_size)
        # status  = 'test less than {0}, do gsea; gmt file use: {1}'.format(each_split_num,gmt_file)
        # print(status)

        if status :
            print('GSEApy successed on {0} gmt!'.format(gmt_file))
        else:
            print('GSEApy failed on {0} gmt!'.format(gmt_file))
    else:
        print('GSEApy will crash, I will split the gmt file by {0} lines or less'.format(each_split_num))

        split_cmd = 'split -l {0} {1} {2}'.format(each_split_num, gmt_file,temp_file_format) 

        exit_status = subprocess.call(split_cmd, shell=True)
    
        if exit_status == 1:
            print('large gmt file "{0}" failed to split. Oh no, Barbecueeeeee le....'.format(gmt_file))
        if exit_status == 0:
            print('large gmt file "{0}" was successed split into {1} lines each file, great!'.format(gmt_file,each_split_num))

        # split command will add nasty suffix to file extension, so need to reformat it
        # os.rename('a.txt', 'b.kml')
        temp_gmt_files = glob.glob('./'+temp_file_format+'??')
        for temp_gmt in temp_gmt_files:
            nasty_suffix = temp_gmt.split(file_name)[-1]
            # print('nasty suffix: {0}'.format(nasty_suffix))
            os.rename(temp_gmt, 'temp_'+nasty_suffix+'_'+file_name)
            # print('new file name: {0}'.format('temp_'+nasty_suffix+'_'+file_name))

        # after split gmt files to temp file, perform gsea analysis
        temp_gmt_files = glob.glob('./temp_*.gmt')

        total_temp_gmt_files = len(temp_gmt_files)

        logging.basicConfig(level=logging.INFO)
        logger = logging.getLogger('now handling {0} gmt files'.format(gmt_file))

        with alive_bar(total_temp_gmt_files) as bar:
        
            for temp_gmt in temp_gmt_files:
                gsea_res, status =  do_gseapy(deg_files, temp_gmt,min_gene_set_size,max_gene_set_size)
                # status  = 'test more than {0}, do gsea; gmt file use: {1}'.format(each_split_num,temp_gmt)
                # print(status)

                if status == True :
                    print('GSEApy successed on {0} gmt!'.format(temp_gmt))
                else:
                    print('GSEApy failed on {0} gmt!'.format(temp_gmt))

                bar()    

        # # rm temp_ gmt_file
        rm_temp_cmd = 'rm {0}'.format('./temp_*') 
        rm_exit_status = subprocess.call(rm_temp_cmd, shell=True)
        if exit_status == 1:
            print('temp gmt file failed to delete')
        if exit_status == 0:
            print('temp gmt files removed')

this_cls: ['shNC_TGFb', 'shNC_TGFb', 'shNC_B14', 'shNC_B14'], using gmt file: AT2_Reyfman_et_al_2019.gmt, using input: shNC-B14vsTGFb
deg file name:             NAME DESCRIPTION  shNC_TGFb_1  shNC_TGFb_2  shNC_B14_1  \
0          TSPAN6          na   481.795098   545.341599  396.204378   
1            DPM1          na   394.712290   404.117679  381.814388   
2           SCYL3          na   139.143181   165.123352  144.859228   
3        C1orf112          na   249.889795   313.951637  253.263815   
4             CFH          na    60.579344   159.691663  326.173095   
...           ...         ...          ...          ...         ...   
11899  AC123768.5          na   644.602085   576.845396  711.824814   
11900  AC019257.7          na   106.013852    80.389001   45.088634   
11901    NIPBL-DT          na    56.793135    74.957311   94.973931   
11902  AL135905.2          na   275.446706   245.512353  296.433784   
11903  AC009090.6          na    52.060374   109.720122   67.153284   


In [ ]:
def drop_empty_folders(directory):
    """Verify that every empty folder removed in local storage."""

    for dirpath, dirnames, filenames in os.walk(directory, topdown=False):
        if not dirnames and not filenames:
            os.rmdir(dirpath)

In [ ]:
drop_empty_folders(gsea_result_path)

In [ ]:
from tqdm.notebook import tqdm
from time import sleep
for i in tqdm(range(100)):
    print(i)
    # sleep(1)

In [ ]:
from alive_progress import alive_bar
import time

for x in 1000, 1500, 700, 0:
   with alive_bar(x) as bar:
       for i in range(1000):
           time.sleep(.005)
           bar()